# **Georgia Disasters Optical HYDRAFloods Code**


**Table of Contents**


1. Overview of HYDRAFloods
2. Set-Up
3. Data Acquisition
5. Optical Imagery Processing and Analysis  
*   Landsat 8 OLI
*   Landsat 7 ETM+
*   Combined Landsat 8 & 7
*   Sentinel 2 MSI






# **1. Overview**
HYDRAFloods - Hydrologic Remote Sensing Analysis for Floods 

HYDRAFloods is a Google Earth Engine (GEE) with Python API based application developed by NASA SERVIR. The application uses satellite imagery to produce flood water maps which can aid in response to flood related disasters. 

This tutorial will walk-through the steps to apply HYDRAFloods to optical imagery to create flood water maps. For this example, we will examine 15 Georgia counties during Hurricanes Irma in September of 2017. 

Study Location: Berrien, Camden, Charlton, Chatham, Coffee, Cook, Crisp, Dougherty, Glynn, Liberty, McIntosh, Thomas, Turner, Wilcox, and Worth counties, GA

This tutorial was developed as part of the Georgia Disasters Project for the NASA DEVELOP Program during the Fall 2022 term. 

The complete documentation for HYDRAFloods can be found [here.](https://servir-mekong.github.io/hydra-floods/)



# **2. Set-Up**

In [1]:
# Mount the drive, authenticate and save credentials
# Click the link, select the appropriate Google Drive Account, and copy the authorization token. Paste the token into the box below.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install HYDRAFloods using Python package installer (pip)
# when this step completes, it will show a warning that you will need to restart. Please ignore this warning.
!pip install hydrafloods geemap rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.6 MB/s 
     |████████████████████████████████| 2.1 MB 15.1 MB/s 
     |████████████████████████████████| 19.3 MB 5.6 MB/s 
     |████████████████████████████████| 87 kB 6.4 MB/s 
     |████████████████████████████████| 99 kB 10.5 MB/s 
     |████████████████████████████████| 198 kB 62.5 MB/s 
     |████████████████████████████████| 1.2 MB 55.9 MB/s 
     |████████████████████████████████| 130 kB 50.6 MB/s 
     |████████████████████████████████| 3.0 MB 42.3 MB/s 
     |████████████████████████████████| 8.8 MB 43.5 MB/s 
     |████████████████████████████████| 1.3 MB 47.8 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 98 kB 9.8 MB/s 
     |████████████████████████████████| 3.7 MB 49.4 MB/s 
     |████████████████████████████████| 46 kB 4.8 MB/s 
     |████████████████████████████████| 12

In [1]:
# loading imports for the program
import ee          
import datetime
import hydrafloods as hf
import geemap.eefolium as geemap
import geemap.colormaps as cm

In [3]:
# authenticate GEE Account
# Click the link, select your GEE Account, copy the token, and paste it into the box below.
_ = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=whhiS9GJp77HpYuWCrq3hnTludBWxMd6m_lMKTaO4Lw&tc=VaDqgR5XtNGBLY_bxoH5lGrpSEbZCV7GZQ8MnITly4Q&cc=yMKvqVmHSQD1nKO1zyoFWUINHR4G1EvETKyc8lyZ-AQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvtD5F9V0zl57IuZuhioOIqq1qerYTzKDfOOoOsMu5y3fTpYpG3C_Ik

Successfully saved authorization token.


# **3. Data Acquisition**

In [5]:
# define region of interest, using shapefile with one feature
region = ee.FeatureCollection("projects/ee-tesfayinathan/assets/15counties").geometry()

In [6]:
# define time period
# for this example, we will use September 10 - 21, 2017 corresponding to Hurricanes Irma
startTime = datetime.datetime(2017, 9, 10);
endTime = datetime.datetime(2017, 9, 21);

In [7]:
# getting optical imagery from Landsat 8 based on time period & region
ls8 = hf.Landsat8(region, startTime, endTime);

In [8]:
# checking for number of images we were able to retrieve from Landsat 8
ls8.n_images

4

In [9]:
# getting optical imagery from Landsat 7 based on time period & region
ls7 = hf.Landsat7(region, startTime, endTime);

In [10]:
# checking number of images we were able to retrieve from Landsat 7
ls7.n_images

2

In [ ]:
ls7.dates

['2017-09-20 16:09:33.380', '2017-09-20 16:09:57.270']

In [ ]:
# getting optical imagery from Sentinel-2 MSI based on time period & region
s2 = hf.Sentinel2(region, startTime, endTime);

In [ ]:
# checking number of images we were able to retrieve from Sentinel-2 MSI
s2.n_images

0

# **5. Optical Imagery Processing and Analysis**

## Landsat 8

In [ ]:
#appy mode reducer to image collection
ls8_image = ls8.collection.reduce('mode')

In [ ]:
#rename bands for MNDWI function
ls8bands = ls8_image.select(['blue_mode', 'green_mode', 'red_mode', 'nir_mode', 'swir1_mode', 'swir2_mode']).rename(
    ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])

In [ ]:
#get mNDWI (modified Normalized Difference Water Index)
ls8_mndwi = hf.mndwi(ls8bands)

In [ ]:
#apply water thresholding algorithm edge otsu, using 300m buffer and 30m scale
landsat8_water = hf.edge_otsu(ls8_mndwi, region = region, edge_buffer = 300, scale = 300, invert = True, thresh_no_data =0.0)

In [ ]:
#retrieve last 5 years of permanent water to compare against current water
#this will allow us to differentiate between flood water and permanent surface water
#we use the JRC dataset to retrieve the 'permanent water'
permanent_water_ls8 = (
    ee.ImageCollection("JRC/GSW1_2/YearlyHistory") # get the JRC historical dataset
    .filterDate("1985-01-01",endTime) # filter for historical data up to date of interest
    .limit(5, "system:time_start", False) # grab the 5 latest images
    .map(lambda x: x.select("waterClass").eq(3)) # extract out the permanent water class
    .sum() # check if a pixel has been classified as permanent water in the past 5 years
    .unmask(0)
    .gt(0)
).updateMask(landsat8_water.gte(0)) # mask for only the water image we just calculated

In [ ]:
#compare our landsat image water detected vs the permanent water map to differentiate flood water 
# from permanent surface water
floods_ls8 = landsat8_water.add(permanent_water_ls8).eq(1).selfMask()

### Full Landsat 8 Map

In [ ]:
#map results/ visualize results
Map = geemap.Map(center=(32, -82.8297), zoom=8)

#Add permanent water layer to map, clip map to region of interest
Map.addLayer(permanent_water_ls8.selfMask(),{"min":0, "max":1,"palette": cm.palettes.Blues}, 'Permanent Water map') 

#Add flood water layer to map, clip map to region of interest
Map.addLayer(floods_ls8.selfMask(),{"min":0, "max":1,"palette":cm.palettes.Reds}, 'Flood map')

#Add county layers
Map.addLayer(region)

Map.addLayerControl()
Map

### Clipped Landsat 8 Map

In [ ]:
#map results/ visualize results
Map = geemap.Map(center=(32, -82.8297), zoom=8)

#Add permanent water layer to map, clip map to region of interest
Map.addLayer(permanent_water_ls8.selfMask().clip(region),{"min":0, "max":1,"palette": cm.palettes.Blues}, 'Permanent Water map') 

#Add flood water layer to map, clip map to region of interest
Map.addLayer(floods_ls8.selfMask().clip(region),{"min":0, "max":1,"palette":cm.palettes.Reds}, 'Flood map')

#Add county polygon layer
Map.addLayer(region)

Map.addLayerControl()
Map

In [ ]:
task = ee.batch.Export.image.toDrive(image=floods_ls8,
                                     description='ls8',
                                     scale=30,
                                     region= region,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')

In [ ]:
task = ee.batch.Export.image.toDrive(image=permanent_water_ls8,
                                     description='permanent_water',
                                     scale=30,
                                     region= region,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')

In [ ]:
# export image using HYDRAFloods export image function. below is function with its parameters
# if the export times out or is taking too long, try exporting at a larger scale (ex 90m)

# hf.export_image(floods_ls8, region, description = "clipped_landsat_8", scale=30, crs='EPSG:4326', pyramiding=".default:mode", export_type='toDrive')

# export with visualizations attached to it - ask Vanessa 
# arc, export map layout, export as pdf/png/jpg

In [ ]:
# Export to asset example
# hf.export_image(floods_ls8, region, description = "clipped_landsat_8", scale=30,  crs='EPSG:4326', pyramiding={"water":"mode"}, export_type='toAsset', asset_id = "projects/ichittumuri/assets/studyarea")

## Landsat 7

In [ ]:
#appy mode reducer to image collection
ls7_image = ls7.collection.reduce('mode')

In [ ]:
#rename bands for MNDWI function
ls7bands = ls7_image.select(['blue_mode', 'green_mode', 'red_mode', 'nir_mode', 'swir1_mode', 'swir2_mode']).rename(
    ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])

In [ ]:
#get mNDWI (modified Normalized Difference Water Index)
ls7_mndwi = hf.mndwi(ls7bands)

In [ ]:
#apply water thresholding algorithm edge otsu, using 300m buffer and 30m scale
landsat7_water = hf.edge_otsu(ls7_mndwi, region = region, edge_buffer = 300, scale = 300, invert = True, thresh_no_data =0.0)

In [ ]:
#retrieve last 5 years of permanent water to compare against current water
#this will allow us to differentiate between flood water and permanent surface water
#we use the JRC dataset to retrieve the 'permanent water'
permanent_water_ls7 = (
    ee.ImageCollection("JRC/GSW1_2/YearlyHistory") # get the JRC historical dataset
    .filterDate("1985-01-01",endTime) # filter for historical data up to date of interest
    .limit(5, "system:time_start", False) # grab the 5 latest images
    .map(lambda x: x.select("waterClass").eq(3)) # extract out the permanent water class
    .sum() # check if a pixel has been classified as permanent water in the past 5 years
    .unmask(0)
    .gt(0)
).updateMask(landsat7_water.gte(0)) # mask for only the water image we just calculated

In [ ]:
#compare our landsat image water detected vs the permanent water map to differentiate flood water 
# from permanent surface water
floods_ls7 = landsat7_water.add(permanent_water_ls7).eq(1).selfMask()

### Full Landsat 7 Map

In [ ]:
#map results/ visualize results
Map = geemap.Map(center=(32, -82.8297), zoom=8)

#Add permanent water layer to map
Map.addLayer(permanent_water_ls7.selfMask(),{"min":0, "max":1,"palette": cm.palettes.Blues}, 'Permanent Water map') 

#Add flood water layer to map
Map.addLayer(floods_ls7.selfMask(),{"min":0, "max":1,"palette":cm.palettes.Reds}, 'Flood map')

#Add county layers
Map.addLayer(region)

Map.addLayerControl()
Map

### Clipped Landsat 7 Map

In [ ]:
#map results/ visualize results
Map = geemap.Map(center=(32, -82.8297), zoom=8)

#Add permanent water layer to map
Map.addLayer(permanent_water_ls7.selfMask().clip(region),{"min":0, "max":1,"palette": cm.palettes.Blues}, 'Permanent Water map') 

#Add flood water layer to map
Map.addLayer(floods_ls7.selfMask().clip(region),{"min":0, "max":1,"palette":cm.palettes.Reds}, 'Flood map')

#Add county layers
Map.addLayer(region)

Map.addLayerControl()
Map

In [ ]:
# export image using HYDRAFloods export image function. below is function with its parameters
# if the export times out or is taking too long, try exporting at a larger scale (ex 90m)
# can change export_type = 'toAsset'

# hf.export_image(floods_ls7, region, description = "clipped_landsat_7", scale=30, crs='EPSG:4326', pyramiding=".default:mode", export_type='toDrive')

In [ ]:
task = ee.batch.Export.image.toDrive(image=floods_ls7,
                                     description='ls7',
                                     scale=30,
                                     region= region,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')

## Combined Clipped Landsat 8 & 7 Map

In [ ]:
Map = geemap.Map(center=(32, -82.8297), zoom=8)

#Add permanent water layer to map, clip map to region of interest
Map.addLayer(permanent_water_ls8.selfMask().clip(region),{"min":0, "max":1,"palette": cm.palettes.Blues}, 'Permanent Water map') 

#Add flood water layer to map, clip map to region of interest
Map.addLayer(floods_ls8.selfMask().clip(region),{"min":0, "max":1,"palette":cm.palettes.Reds}, 'Landsat 8 Flood map')

#Add flood water layer to map, clip map to region of interest
Map.addLayer(floods_ls7.selfMask().clip(region),{"min":0, "max":1,"palette":cm.palettes.Reds}, 'Landsat 7 Flood map')

#Add county layers
Map.addLayer(region)

Map.addLayerControl()
Map

In [ ]:
# export image using HYDRAFloods export image function. below is function with its parameters
# if the export times out or is taking too long, try exporting at a larger scale (ex 90m)
# can change export_type = 'toAsset'

# hf.export_image(floods_ls7, region, description = "clipped_combined_ls7&8", scale=30, crs='EPSG:4326', pyramiding=".default:mode", export_type='toDrive')

In [ ]:
task = ee.batch.Export.image.toDrive(image=floods_ls8,
                                     description='ls8',
                                     scale=30,
                                     region= region,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()

In [ ]:
task.status()

{'state': 'READY',
 'description': 'ls8',
 'creation_timestamp_ms': 1667323986338,
 'update_timestamp_ms': 1667323986338,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '2VT7XB6SOWBWG2D45PB6EG54',
 'name': 'projects/earthengine-legacy/operations/2VT7XB6SOWBWG2D45PB6EG54'}

## Sentinel 2